# Data pre-processing

In [1]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os # accessing directory structure

In [3]:
DATASET_PATH = "myntradataset/"
print(os.listdir(DATASET_PATH))

['styles.csv', 'images']


In [4]:
image_meta = pd.read_csv(DATASET_PATH+'styles.csv',error_bad_lines=False)

b'Skipping line 6044: expected 10 fields, saw 11\nSkipping line 6569: expected 10 fields, saw 11\nSkipping line 7399: expected 10 fields, saw 11\nSkipping line 7939: expected 10 fields, saw 11\nSkipping line 9026: expected 10 fields, saw 11\nSkipping line 10264: expected 10 fields, saw 11\nSkipping line 10427: expected 10 fields, saw 11\nSkipping line 10905: expected 10 fields, saw 11\nSkipping line 11373: expected 10 fields, saw 11\nSkipping line 11945: expected 10 fields, saw 11\nSkipping line 14112: expected 10 fields, saw 11\nSkipping line 14532: expected 10 fields, saw 11\nSkipping line 15076: expected 10 fields, saw 12\nSkipping line 29906: expected 10 fields, saw 11\nSkipping line 31625: expected 10 fields, saw 11\nSkipping line 33020: expected 10 fields, saw 11\nSkipping line 35748: expected 10 fields, saw 11\nSkipping line 35962: expected 10 fields, saw 11\nSkipping line 37770: expected 10 fields, saw 11\nSkipping line 38105: expected 10 fields, saw 11\nSkipping line 38275: ex

In [5]:
image_meta.shape

(44424, 10)

In [6]:
image_meta.head()

,id,gender,masterCategory,subCategory,articleType,baseColour,season,year,usage,productDisplayName
0,15970,Men,Apparel,Topwear,Shirts,Navy Blue,Fall,2011.0,Casual,Turtle Check Men Navy Blue Shirt
1,39386,Men,Apparel,Bottomwear,Jeans,Blue,Summer,2012.0,Casual,Peter England Men Party Blue Jeans
2,59263,Women,Accessories,Watches,Watches,Silver,Winter,2016.0,Casual,Titan Women Silver Watch
3,21379,Men,Apparel,Bottomwear,Track Pants,Black,Fall,2011.0,Casual,Manchester United Men Solid Black Track Pants
4,53759,Men,Apparel,Topwear,Tshirts,Grey,Summer,2012.0,Casual,Puma Men Grey T-shirt


In [7]:
image_meta['image'] = image_meta.apply(lambda row: str(row['id']) + ".jpg", axis=1)

In [8]:
train_image_meta = image_meta.groupby('articleType').head(100)

In [9]:
train_image_meta.head(5)

,id,gender,masterCategory,subCategory,articleType,baseColour,season,year,usage,productDisplayName,image
0,15970,Men,Apparel,Topwear,Shirts,Navy Blue,Fall,2011.0,Casual,Turtle Check Men Navy Blue Shirt,15970.jpg
1,39386,Men,Apparel,Bottomwear,Jeans,Blue,Summer,2012.0,Casual,Peter England Men Party Blue Jeans,39386.jpg
2,59263,Women,Accessories,Watches,Watches,Silver,Winter,2016.0,Casual,Titan Women Silver Watch,59263.jpg
3,21379,Men,Apparel,Bottomwear,Track Pants,Black,Fall,2011.0,Casual,Manchester United Men Solid Black Track Pants,21379.jpg
4,53759,Men,Apparel,Topwear,Tshirts,Grey,Summer,2012.0,Casual,Puma Men Grey T-shirt,53759.jpg


In [10]:
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array, array_to_img

Using TensorFlow backend.


In [11]:
delete_row = train_image_meta[train_image_meta["id"]==12347].index
train_image_meta = train_image_meta.drop(delete_row)

In [12]:
IMG_WIDTH=224
IMG_HEIGHT=224
IMG_DIM = (IMG_WIDTH, IMG_HEIGHT)
IMG_PATH = DATASET_PATH + 'images/'

train_imgs = [img_to_array(load_img(IMG_PATH+img.image, target_size=IMG_DIM)) for i, img in train_image_meta.iterrows()]
train_imgs = np.array(train_imgs)
train_labels = [img.articleType for i, img in train_image_meta.iterrows()]

In [13]:
# encode text category labels 
from sklearn.preprocessing import LabelEncoder 
 
le = LabelEncoder() 
le.fit(train_labels) 
train_labels_enc = le.transform(train_labels) 

In [14]:
train_labels_enc

array([104,  56, 139, ...,  44,  37,  52])

In [15]:
name_labels = [(train_labels[i], train_labels_enc[i]) for i in range(0, len(train_labels_enc))]
    

In [16]:
from collections import Counter

input_shape = (IMG_WIDTH, IMG_HEIGHT)
NUM_CLASSES = len(Counter(train_labels_enc).keys())

In [17]:
NUM_CLASSES

142

In [18]:
train_labels_enc.shape

(7669,)

In [19]:
from keras.utils import to_categorical
train_labels_enc = to_categorical(train_labels_enc)

In [20]:
train_labels_enc.shape

(7669, 142)

In [21]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                    shear_range=0.2,
                                    zoom_range=0.2,
                                    horizontal_flip=True,
                                    validation_split=0.2)

In [22]:
train_generator = train_datagen.flow(train_imgs, 
                                     train_labels_enc,
                                     batch_size=30, 
                                     subset='training',
                                     shuffle=True)

In [23]:
validation_generator = train_datagen.flow(train_imgs, 
                                     train_labels_enc,
                                     batch_size=30, 
                                     subset='validation',
                                     shuffle=True)

In [24]:
from keras.applications.resnet50 import ResNet50
from keras.models import Model
import keras
restnet = ResNet50(include_top=False, weights='imagenet', input_shape=(IMG_HEIGHT,IMG_WIDTH,3))
# output = restnet.layers[-1].output
# output = keras.layers.Flatten()(output)
# restnet = Model(restnet.input, output=output)
for layer in restnet.layers:
    layer.trainable = False
restnet.summary()

/home/bjit/AI/Jupyter/env/lib/python3.6/site-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
___________________________________________________________________________________________

In [25]:
restnet.trainable = True
set_trainable = False
for layer in restnet.layers:
    if layer.name in ['res5c_branch2b', 'res5c_branch2c', 'activation_97']:
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False

In [26]:
from keras.layers import Conv2D, GlobalAveragePooling2D, Flatten, Dense, Dropout, InputLayer
# from keras.models import Sequential
from keras import optimizers
# model_finetuned = Sequential()
# model_finetuned.add(restnet)
# model_finetuned.add(Dense(512, activation='relu'))
# model_finetuned.add(Dropout(0.3))
# model_finetuned.add(Dense(NUM_CLASSES, activation='softmax'))
# model_finetuned.compile(loss='categorical_crossentropy',
#               optimizer=optimizers.RMSprop(lr=1e-5),
#               metrics=['accuracy'])
# model_finetuned.summary()

In [27]:
base_model = restnet
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
# and a logistic layer -- let's say we have 7 classes
predictions = Dense(NUM_CLASSES, activation='softmax')(x) 
model_finetuned = Model(inputs=base_model.input, outputs=predictions)
model_finetuned.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-5),
              metrics=['accuracy'])
model_finetuned.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
____________________________________________________________________________________________

In [28]:

os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [31]:
history_1 = model_finetuned.fit_generator(train_generator, 
                                          steps_per_epoch=None, 
                                          epochs=5, 
                                          verbose=1, 
                                          use_multiprocessing=True,
                                          validation_data = validation_generator,
                                          workers=1)

Epoch 1/5
 75/205 [=========>....................] - ETA: 2:49 - loss: 5.0074 - accuracy: 0.0134

Process ForkPoolWorker-26:
Process ForkPoolWorker-25:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.6/multiprocessing/queues.py", line 335, in get
    res = self._reader.recv_bytes()
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 216, in recv_bytes
    buf = self._recv_bytes(maxlength)
  File "/usr/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/usr/lib/python3.6/multiprocessin

  File "/usr/lib/python3.6/multiprocessing/connection.py", line 216, in recv_bytes
    buf = self._recv_bytes(maxlength)
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 407, in _recv_bytes
    buf = self._recv(4)
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 379, in _recv
    chunk = read(handle, remaining)
KeyboardInterrupt


Exception in thread Thread-10:
Traceback (most recent call last):
  File "/usr/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.6/multiprocessing/pool.py", line 405, in _handle_workers
    pool._maintain_pool()
  File "/usr/lib/python3.6/multiprocessing/pool.py", line 246, in _maintain_pool
    self._repopulate_pool()
  File "/usr/lib/python3.6/multiprocessing/pool.py", line 239, in _repopulate_pool
    w.start()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 105, in start
    self._popen = self._Popen(self)
  File "/usr/lib/python3.6/multiprocessing/context.py", line 277, in _Popen
    return Popen(process_obj)
  File "/usr/lib/python3.6/multiprocessing/popen_fork.py", line 19, in __init__
    self._launch(process_obj)
  File "/usr/lib/python3.6/multiprocessing/popen_fork.py", line 66, in _launch
    self.pid = os.fork

KeyboardInterrupt: 

In [ ]:
from PIL import Image
import requests
from io import BytesIO

response = requests.get(query_image_path)
img = Image.open(BytesIO(response.content))

In [ ]:
import cv2
img = cv2.imread(query_image_path)
# query_image = cv2.resize(img, IMG_DIM);

In [ ]:
from keras.applications.resnet50 import preprocess_input, decode_predictions

In [ ]:
from io import BytesIO
import urllib

def loadImage(URL):
    if "http" in URL:
        with urllib.request.urlopen(URL) as url:
            img = load_img(BytesIO(url.read()), target_size=(IMG_WIDTH,IMG_HEIGHT))
    else:
        img = load_img(URL, target_size=IMG_DIM)

    return img

In [ ]:
def get_prediction(model, path):
    # Reshape
    img =  loadImage(path)
    # img to Array
    x   = img_to_array(img)
    # Expand Dim (1, w, h)
    x   = np.expand_dims(x, axis=0)
    # Pre process Input
    x   = preprocess_input(x)
    x   = model.predict(x).reshape(-1)
    result = np.where(x == np.amax(x))
    return [item for item in name_labels if item[1] == result[0]][0]

In [ ]:
query_image_path = 'https://www.threadsmenswear.com/images/versace-collection-logo-badge-print-tshirt-p3709-102353_image.jpg';
prediction = get_prediction(model_finetuned, query_image_path)
print(prediction)

In [ ]:
model_finetuned.save('fashion_resnet.h5')

In [ ]:
prediction.shape

In [ ]:
prediction = list(enumerate(prediction))

In [ ]:
import operator
prediction.sort(key = operator.itemgetter(1))

In [ ]:
prediction[len(prediction)-1]

In [ ]:
prediction.sort(-1)

In [ ]:
np.amax(prediction)

In [ ]:
result = np.where(prediction == np.amax(prediction))

In [ ]:
result[0]

In [ ]:
na[53]

In [ ]:
[item for item in name_labels if item[1] == 19][0]

In [ ]:
items[0]

# End Custom processing

In [ ]:
df = pd.read_csv(DATASET_PATH + "styles.csv", nrows=5000, error_bad_lines=False)
df['image'] = df.apply(lambda row: str(row['id']) + ".jpg", axis=1)
df = df.reset_index(drop=True)
df.head(10)

In [ ]:
df['articleType'].value_counts()

In [ ]:
import cv2
def plot_figures(figures, nrows = 1, ncols=1,figsize=(8, 8)):
    """Plot a dictionary of figures.

    Parameters
    ----------
    figures : <title, figure> dictionary
    ncols : number of columns of subplots wanted in the display
    nrows : number of rows of subplots wanted in the figure
    """

    fig, axeslist = plt.subplots(ncols=ncols, nrows=nrows,figsize=figsize)
    for ind,title in enumerate(figures):
        axeslist.ravel()[ind].imshow(cv2.cvtColor(figures[title], cv2.COLOR_BGR2RGB))
        axeslist.ravel()[ind].set_title(title)
        axeslist.ravel()[ind].set_axis_off()
    plt.tight_layout() # optional
    
def img_path(img):
    return DATASET_PATH+"/images/"+img

def load_image(img, resized_fac = 0.1):
    img     = cv2.imread(img_path(img))
    w, h, _ = img.shape
    resized = cv2.resize(img, (int(h*resized_fac), int(w*resized_fac)), interpolation = cv2.INTER_AREA)
    return resized

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# generation of a dictionary of (title, images)
figures = {'im'+str(i): load_image(row.image) for i, row in df.sample(6).iterrows()}
# plot of the images in a figure, with 2 rows and 3 columns
plot_figures(figures, 2, 3)

The Dataset is made up of different items that can be found in a marketplace. The idea is to use embeddings to search for similarity and find similar items just using the image.

In [ ]:
plt.figure(figsize=(7,20))
df.articleType.value_counts().sort_values().plot(kind='barh')

## Use Pre-Trained Model to Recommendation

In [ ]:
import tensorflow as tf
import keras
from keras import Model
from keras.applications.xception import Xception
from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.applications.xception import preprocess_input as xcept_preprocess_input, decode_predictions as xcept_decode_predictions
from keras.applications.resnet50 import preprocess_input as resn_preprocess_input, decode_predictions as resn_decode_predictions
from keras.layers import GlobalMaxPooling2D
from keras.layers import Dense
from keras.models import Sequential
tf.__version__

In [ ]:
# Fixed for our Cats & Dogs classes
NUM_CLASSES = 108

# Fixed for Cats & Dogs color images
CHANNELS = 3

IMAGE_RESIZE = 224
RESNET50_POOLING_AVERAGE = 'avg'
DENSE_LAYER_ACTIVATION = 'softmax'
OBJECTIVE_FUNCTION = 'categorical_crossentropy'

# Common accuracy metric for all outputs, but can use different metrics for different output
LOSS_METRICS = ['accuracy']

# EARLY_STOP_PATIENCE must be < NUM_EPOCHS
NUM_EPOCHS = 10
EARLY_STOP_PATIENCE = 3

# These steps value should be proper FACTOR of no.-of-images in train & valid folders respectively
# Training images processed in each step would be no.-of-train-images / STEPS_PER_EPOCH_TRAINING
STEPS_PER_EPOCH_TRAINING = 10
STEPS_PER_EPOCH_VALIDATION = 10

# These steps value should be proper FACTOR of no.-of-images in train & valid folders respectively
# NOTE that these BATCH* are for Keras ImageDataGenerator batching to fill epoch step input
BATCH_SIZE_TRAINING = 100
BATCH_SIZE_VALIDATION = 100

# Using 1 to easily manage mapping between test_generator & prediction for submission preparation
BATCH_SIZE_TESTING = 1
img_width, img_height, _ = 224, 224, 3 #load_image(df.iloc[0].image).shape

In [ ]:
#Still not talking about our train/test data or any pre-processing.



# 1st layer as the lumpsum weights from resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5
# NOTE that this layer will be set below as NOT TRAINABLE, i.e., use it as is
# model.add(ResNet50(include_top = False, pooling = RESNET50_POOLING_AVERAGE, weights = 'imagenet'))
base_model = ResNet50(weights='imagenet', 
                      include_top=False, 
                      input_shape = (img_width, img_height, 3))
base_model.trainable = False

# 2nd layer as Dense for 2-class classification, i.e., dog or cat using SoftMax activation
output_layer = Dense(NUM_CLASSES, activation = DENSE_LAYER_ACTIVATION)

model = Sequential([
    base_model,
    output_layer
])
# Say not to train first layer (ResNet) model as it is already trained
model.summary()

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

TRAIN_DIR = "/kaggle/input/fashion-product-images-dataset/fashion-dataset/fashion-dataset/"

train_datagen =  ImageDataGenerator(
      preprocessing_function=resn_preprocess_input,
      rotation_range=90,
      horizontal_flip=True,
      vertical_flip=True
    )

train_generator = train_datagen.flow_from_directory(TRAIN_DIR, 
                                                    target_size=(img_height, img_width), 
                                                    batch_size=BATCH_SIZE_TRAINING,
                                                    class_mode='categorical')

In [ ]:
from keras import optimizers

sgd = optimizers.SGD(lr = 0.01, decay = 1e-6, momentum = 0.9, nesterov = True)
model.compile(optimizer = sgd, loss = OBJECTIVE_FUNCTION, metrics = LOSS_METRICS)

### Xception

In [ ]:
# Input Shape
img_width, img_height, _ = 224, 224, 3 #load_image(df.iloc[0].image).shape

# Pre-Trained Model
xcept_base_model = Xception(weights='imagenet',
                      input_shape=(img_width, img_height, 3), 
                      include_top=False)
xcept_base_model.trainable = False

# Add Layer Embedding
xcept_model = keras.Sequential([
    xcept_base_model,
    GlobalMaxPooling2D()
])

xcept_model.summary()

### ResNet50

In [ ]:
# Pre-Trained Model
resn_base_model = ResNet50(weights='imagenet', 
                      include_top=False, 
                      input_shape = (img_width, img_height, 3))
resn_base_model.trainable = False

# Add Layer Embedding
resn_model = keras.Sequential([
    resn_base_model,
    GlobalMaxPooling2D()
])

resn_model.summary()

In [ ]:
def get_embedding(model, img_name, model_name):
    # Reshape
    img = image.load_img(img_path(img_name), target_size=(img_width, img_height))
    # img to Array
    x   = image.img_to_array(img)
    # Expand Dim (1, w, h)
    x   = np.expand_dims(x, axis=0)
    # Pre process Input
    if model_name == 'resnet':
        x = resn_preprocess_input(x)
    else:
        x = xcept_preprocess_input(x)
        
    return model.predict(x).reshape(-1)

Get item Embedding

### Xception embeddings

In [ ]:
xcept_emb = get_embedding(xcept_model, df.iloc[0].image, 'xception')
xcept_emb.shape

In [ ]:
img_array = load_image(df.iloc[0].image)
plt.imshow(cv2.cvtColor(img_array, cv2.COLOR_BGR2RGB))
print(img_array.shape)
print(xcept_emb)

### ResNet50 embedding

In [ ]:
resn_emb = get_embedding(resn_model, df.iloc[0].image, 'resnet')
resn_emb.shape

In [ ]:
img_array = load_image(df.iloc[0].image)
plt.imshow(cv2.cvtColor(img_array, cv2.COLOR_BGR2RGB))
print(img_array.shape)
print(resn_emb)

### Xception embedding mapping

In [ ]:
%%time
#import swifter

# Parallel apply
xcept_df_sample      = df#.sample(10)
xcept_map_embeddings = xcept_df_sample['image'].apply(lambda img: get_embedding(xcept_model, img, 'xception'))
xcept_df_embs        = xcept_map_embeddings.apply(pd.Series)

print(xcept_df_embs.shape)
xcept_df_embs.head()

### ResNet50 embedding mapping

In [ ]:
%%time
#import swifter

# Parallel apply
resn_df_sample      = df#.sample(10)
resn_map_embeddings = resn_df_sample['image'].apply(lambda img: get_embedding(resn_model, img, 'resnet'))
resn_df_embs        = resn_map_embeddings.apply(pd.Series)

print(resn_df_embs.shape)
resn_df_embs.head()

#### Compute Similarity Between Items

![](http://dataaspirant.com/wp-content/uploads/2015/04/cosine.png)

In [ ]:
# https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise_distances.html
from sklearn.metrics.pairwise import pairwise_distances

### Xception pairwise similarity

In [ ]:
# Calcule DIstance Matriz
xcept_cosine_sim = 1-pairwise_distances(xcept_df_embs, metric='cosine')
xcept_cosine_sim[:4, :4]

### ResNet50 pairwise similarity

In [ ]:
# Calcule DIstance Matriz
resn_cosine_sim = 1-pairwise_distances(resn_df_embs, metric='cosine')
resn_cosine_sim[:4, :4]

#### Recommender Similar Items

In [ ]:
indices = pd.Series(range(len(df)), index=df.index)
indices

# Function that get movie recommendations based on the cosine similarity score of movie genres
def get_recommender(idx, df, cosine_sim, top_n = 5):
    sim_idx    = indices[idx]
    sim_scores = list(enumerate(cosine_sim[sim_idx]))
    actual_scores = sim_scores[1:]
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:top_n+1]
    idx_rec    = [i[0] for i in sim_scores]
    idx_sim    = [i[1] for i in sim_scores]
    
    return indices.iloc[idx_rec].index, idx_sim, actual_scores

get_recommender(2993, df, xcept_cosine_sim, top_n = 5)

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

def plot_figure(x_actual, res_actual, yLabel, xLabel, title):
    figure(num=None, figsize=(12, 5), dpi=80, facecolor='w', edgecolor='k')
    plt.plot(*zip(*x_actual)) 
    plt.plot(*zip(*res_actual))
    plt.legend( ('ResNet50','Xception') )
    plt.ylabel(yLabel)
    plt.xlabel(xLabel)
    plt.title(title)
    plt.show()

### Calculate how similar two models are

In [ ]:
def countSimilarity(sim1, sim2):
    same = 0
    for i in range(len(sim1)):
        if i>0 and ((sim1[i][1]-sim1[i-1][1])*(sim2[i][1]-sim2[i-1][1]))>=0:
            same = same + 1
    return (same*100)/(len(sim1)-1)

countSimilarity([(1, 0.39834428), (1, 0.39834428)], [(1, 0.39834428), (1, 0.39834428)])

### Xception suggestions

In [ ]:
# Idx Item to Recommender
idx_ref = 2993

# Recommendations
idx_rec, idx_sim, xcept_actual = get_recommender(idx_ref, df, xcept_cosine_sim , top_n = 6)

# Plot
#===================
plt.imshow(cv2.cvtColor(load_image(df.iloc[idx_ref].image), cv2.COLOR_BGR2RGB))

# generation of a dictionary of (title, images)
figures = {'im'+str(i): load_image(row.image) for i, row in df.loc[idx_rec].iterrows()}
# plot of the images in a figure, with 2 rows and 3 columns
plot_figures(figures, 2, 3)

### ResNet50 suggestions

In [ ]:
# Idx Item to Recommender
idx_ref = 2993

# Recommendations
idx_rec, idx_sim, resn_actual = get_recommender(idx_ref, df, resn_cosine_sim, top_n = 6)

# Plot
#===================
plt.imshow(cv2.cvtColor(load_image(df.iloc[idx_ref].image), cv2.COLOR_BGR2RGB))

# generation of a dictionary of (title, images)
figures = {'im'+str(i): load_image(row.image) for i, row in df.loc[idx_rec].iterrows()}
# plot of the images in a figure, with 2 rows and 3 columns
plot_figures(figures, 2, 3)

In [ ]:
plot_figure(xcept_actual,resn_actual, 'Similarity', 'Product Id' ,'2993')

In [ ]:
countSimilarity(xcept_actual, resn_actual)

**Search for items similar to the reference to recommend. Apparently it's working!**

### Xception suggestions

In [ ]:
idx_ref = 879

# Recommendations
idx_rec, idx_sim, xcept_actual = get_recommender(idx_ref, df, xcept_cosine_sim, top_n = 6)

# Plot
#===================
plt.imshow(cv2.cvtColor(load_image(df.iloc[idx_ref].image), cv2.COLOR_BGR2RGB))

# generation of a dictionary of (title, images)
figures = {'im'+str(i): load_image(row.image) for i, row in df.loc[idx_rec].iterrows()}
# plot of the images in a figure, with 2 rows and 3 columns
plot_figures(figures, 2, 3)

### ResNet50 suggestions

In [ ]:
idx_ref = 879

# Recommendations
idx_rec, idx_sim, resn_actual = get_recommender(idx_ref, df, resn_cosine_sim, top_n = 6)

# Plot
#===================
plt.imshow(cv2.cvtColor(load_image(df.iloc[idx_ref].image), cv2.COLOR_BGR2RGB))

# generation of a dictionary of (title, images)
figures = {'im'+str(i): load_image(row.image) for i, row in df.loc[idx_rec].iterrows()}
# plot of the images in a figure, with 2 rows and 3 columns
plot_figures(figures, 2, 3)

In [ ]:
plot_figure(xcept_actual, resn_actual, 'Similarity', 'Product Id' ,'879')

In [ ]:
countSimilarity(xcept_actual, resn_actual)

### Xception suggestions

In [ ]:
idx_ref = 987

# Recommendations
idx_rec, idx_sim, xcept_actual = get_recommender(idx_ref, df, xcept_cosine_sim, top_n = 6)

# Plot
#===================
plt.imshow(cv2.cvtColor(load_image(df.iloc[idx_ref].image), cv2.COLOR_BGR2RGB))

# generation of a dictionary of (title, images)
figures = {'im'+str(i): load_image(row.image) for i, row in df.loc[idx_rec].iterrows()}
# plot of the images in a figure, with 2 rows and 3 columns
plot_figures(figures, 2, 3)

### ResNet50 suggestions

In [ ]:
idx_ref = 987

# Recommendations
idx_rec, idx_sim, resn_actual = get_recommender(idx_ref, df, resn_cosine_sim, top_n = 6)

# Plot
#===================
plt.imshow(cv2.cvtColor(load_image(df.iloc[idx_ref].image), cv2.COLOR_BGR2RGB))

# generation of a dictionary of (title, images)
figures = {'im'+str(i): load_image(row.image) for i, row in df.loc[idx_rec].iterrows()}
# plot of the images in a figure, with 2 rows and 3 columns
plot_figures(figures, 2, 3)

In [ ]:
plot_figure(xcept_actual ,resn_actual, 'Similarity', 'Product Id' ,'987')

In [ ]:
countSimilarity(xcept_actual, resn_actual)

### Xception suggestions

In [ ]:
idx_ref = 3524

# Recommendations
idx_rec, idx_sim, xcept_actual = get_recommender(idx_ref, df, xcept_cosine_sim,  top_n = 6)

# Plot
#===================
plt.imshow(cv2.cvtColor(load_image(df.iloc[idx_ref].image), cv2.COLOR_BGR2RGB))

# generation of a dictionary of (title, images)
figures = {'im'+str(i): load_image(row.image) for i, row in df.loc[idx_rec].iterrows()}
# plot of the images in a figure, with 2 rows and 3 columns
plot_figures(figures, 2, 3)

### ResNet50 suggestions

In [ ]:
idx_ref = 3524

# Recommendations
idx_rec, idx_sim, resn_actual = get_recommender(idx_ref, df, resn_cosine_sim,  top_n = 6)

# Plot
#===================
plt.imshow(cv2.cvtColor(load_image(df.iloc[idx_ref].image), cv2.COLOR_BGR2RGB))

# generation of a dictionary of (title, images)
figures = {'im'+str(i): load_image(row.image) for i, row in df.loc[idx_rec].iterrows()}
# plot of the images in a figure, with 2 rows and 3 columns
plot_figures(figures, 2, 3)

In [ ]:
countSimilarity(xcept_actual, resn_actual)

In [ ]:
plot_figure(xcept_actual, resn_actual, 'Similarity', 'Product Id' ,'3524')